In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

# 1. 현재가 가져오기 (선택 코인)
def get_ticker_data(markets):
    url = f"https://api.upbit.com/v1/ticker?markets={','.join(markets)}"
    res = requests.get(url).json()
    rows = []
    ts = datetime.now().replace(second=0, microsecond=0)  # 정각 timestamp
    for coin in res:
        row = {
            "timestamp": ts,
            "market": coin["market"],
            "trade_price": coin["trade_price"],
            "change_rate": coin["signed_change_rate"],
            "acc_trade_volume": coin["acc_trade_volume"]
        }
        rows.append(row)
    return pd.DataFrame(rows)

# 2. 파이프라인 실행
def run_pipeline(hours=12):
    markets = ["KRW-BTC", "KRW-ETH", "KRW-DOGE", "KRW-XRP"]
    all_data = []
    end_time = datetime.now() + pd.Timedelta(hours=hours)
    
    while datetime.now() < end_time:
        now = datetime.now()
        if now.minute == 0:  # 정각에만 수집
            df = get_ticker_data(markets)
            all_data.append(df)
            print(df)
            # 다음 정각까지 sleep
            time_to_next_hour = 60*60 - now.minute*60 - now.second
            time.sleep(time_to_next_hour)
        else:
            # 정각까지 대기
            time_to_next_hour = (60 - now.minute) * 60 - now.second
            time.sleep(time_to_next_hour)
    
    # 결과 저장
    result = pd.concat(all_data, ignore_index=True)
    result.to_csv("upbit_selected_coins_hourly.csv", index=False, encoding="utf-8-sig")
    print("데이터 수집 완료, CSV 저장됨")
    return result

if __name__ == "__main__":
    df = run_pipeline(hours=24)

                   timestamp      market  trade_price  change_rate  \
0 2025-09-22 11:37:33.950083    KRW-WAXP         26.3    -0.025926   
1 2025-09-22 11:37:33.950094    KRW-CARV        419.0    -0.049887   
2 2025-09-22 11:37:33.950095     KRW-LSK        481.0    -0.030242   
3 2025-09-22 11:37:33.950097    KRW-BORA        120.0    -0.024390   
4 2025-09-22 11:37:33.950098  KRW-PUNDIX        423.0    -0.029817   

   acc_trade_volume  
0      1.051872e+07  
1      2.166007e+06  
2      8.177762e+05  
3      3.901531e+06  
4      7.690343e+05  
                   timestamp      market  trade_price  change_rate  \
0 2025-09-22 11:37:44.241621    KRW-WAXP         26.3    -0.025926   
1 2025-09-22 11:37:44.241631    KRW-CARV        419.0    -0.049887   
2 2025-09-22 11:37:44.241633     KRW-LSK        481.0    -0.030242   
3 2025-09-22 11:37:44.241634    KRW-BORA        121.0    -0.016260   
4 2025-09-22 11:37:44.241635  KRW-PUNDIX        423.0    -0.029817   

   acc_trade_volume  
0   